<a href="https://colab.research.google.com/github/rypotter/depo/blob/master/ImplementHuggingFaceModelsUsingLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Comprehensive Guide to Implement HuggingFace Models Using Langchain

https://www.analyticsvidhya.com/blog/2023/12/implement-huggingface-models-using-langchain/

In [ ]:
!pip install langchain

In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install bitsandbytes accelerate
#!pip install -i https://test.pypi.org/simple/ bitsandbytes-cuda112
#!pip install accelerate

In [ ]:
!pip install llama-cpp-python

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install spicy

# **Approach 1: HuggingFace Pipeline**

In [8]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


In [9]:
model_id = "pankajmathur/orca_mini_3b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 quantization_config=nf4_config
                 )
pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               max_new_tokens=512
               )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [14]:
hf = HuggingFacePipeline(pipeline=pipe)

query = "Who is Shah Rukh Khan?"

prompt = f"""
### System:
You are an AI assistant that follows instruction extremely well.
Help as much as you can. Please be truthful and give direct answers

### User:
{query}

### Response:
"""

response = hf.predict(prompt)
print(response)

 Shah Rukh Khan is a popular Indian actor, producer, and television personality who has worked in Bollywood films since the 1990s. He is known for his unique style, charming personality, and powerful on-screen presence. He has won several awards, including a National Film Award and five Filmfare Awards. He is also known for his philanthropic work and has been involved in various social causes.


In [ ]:
# hf = HuggingFacePipeline(pipeline=pipe)

# query = "Who is Shah Rukh Khan?"

query = "write a python script which copies a directory to a new directory and replaces a string with another string in all files with a given extension in the new directory and its subdirectories"

prompt = f"""
### System:
You are an AI assistant that follows instruction extremely well.
Help as much as you can. Please be truthful and give direct answers

### User:
{query}

### Response:
"""

response = hf.predict(prompt)
print(response)

# **Approach 2: HuggingFace Hub using Inference API**

In approach one, you might have noticed that while using the pipeline, the model and tokenization download and load the weights. This approach might be time-consuming if the length of the model is enormous. Thus, the HuggingFace Hub Inference API comes in handy. To integrate HuggingFace Hub with Langchain, one requires a HuggingFace Access Token.

Steps to get HuggingFace Access Token

Log in to HuggingFace.co.
Click on your profile icon at the top-right corner, then choose “Settings.”
In the left sidebar, navigate to “Access Token.”
Generate a new access token, assigning it the “write” role.

# **Approach 3: LlamaCPP**

LLamaCPP allows the use of models packaged as. gguf files format that runs efficiently in CPU-only and mixed CPU/GPU environments using the llama.

To use LlamaCPP, we specifically need models whose model_path ends with gguf. You can download the model from here: zephyr-7b-beta.Q4.gguf. Once this model is downloaded, you can directly upload it to your drive or any other local storage.